In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
# Set up Chrome with options
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")  # Avoid bot detection
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")  # Fake user-agent
options.add_argument("--incognito")  # Open in incognito mode

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



In [30]:
def weworkremotelyJobs(title): 
    url = f"https://weworkremotely.com/remote-jobs/search?search_uuid=&term={title}&button=&sort=past_week" 
    driver.get(url) 
    wait = WebDriverWait(driver, 10)  
     
    try: 
        jobs_list = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="category-17"]/article/ul'))) 
        job_items = jobs_list.find_elements(By.TAG_NAME, 'li')  
        print(f"Total jobs found: {len(job_items)}") 
        jobs = [] 
     
        for job in job_items:  
            try: 
                if "view-all" in job.get_attribute('class'): 
                    continue 
                
                href = job.find_element(By.TAG_NAME, 'a').get_attribute('href') 
                title = job.find_element(By.CLASS_NAME, 'title').text 
                company_info = job.find_elements(By.CLASS_NAME, 'company')
                company = company_info[0].text if len(company_info) > 0 else "Unknown"
                job_type = company_info[1].text if len(company_info) > 1 else "Not specified"
                # Handle missing location
                try:
                    location = job.find_element(By.CLASS_NAME, 'region.company').text 
                except:
                    location = "Not specified"
                
                # Handle missing posted date
                try:
                    posted_elements = job.find_elements(By.CLASS_NAME, 'listing-date__date')
                    posted = posted_elements[0].text if posted_elements else "Not provided"
                except:
                    posted = "Not provided"

                # Handle missing image
                try:
                    img = job.find_element(By.CLASS_NAME, 'flag-logo').get_attribute('style') 
                    start = img.find('url("')+5  # Find start of URL
                    img = img[start:len(img)-3]
                except:
                    img = "No image"

                job_as_JSON = { 
                    "title": title, 
                    "company": company, 
                    "location": location, 
                    "posted": posted, 
                    "href": href, 
                    "img": img,
                    "type":job_type
                } 
                jobs.append(job_as_JSON) 
                print(f"Title: {title}\nCompany: {company}\nLocation: {location}\nPosted: {posted}\nLink: {href}\nImage: {img}\n") 
 
            except Exception as e: 
                print(f"Error extracting job details: {e}") 

        with open("weworkremotely_jobs.json", "w", encoding="utf-8") as f: 
            json.dump(jobs, f, indent=4, ensure_ascii=False) 

    except Exception as e: 
        print("Job listings not found or error:", e) 

weworkremotelyJobs('Engineer') 

Total jobs found: 6
Title: Software Engineer (Python), Data Platform
Company: Doximity
Location: USA Only
Posted: 2d
Link: https://weworkremotely.com/company/doximity
Image: No image

Title: Senior Frontend Engineer (React, TypeScript)
Company: Paperpile
Location: Anywhere in the World
Posted: 4d
Link: https://weworkremotely.com/company/paperpile
Image: https://we-work-remotely.imgix.net/logos/0162/2481/logo.gif?ixlib=rails-4.0.0&w=50&h=50&dpr=2&fit=fill&auto=compress

Title: Senior Mobile Engineer (iOS and Android)
Company: Paperpile
Location: Anywhere in the World
Posted: 4d
Link: https://weworkremotely.com/company/paperpile
Image: https://we-work-remotely.imgix.net/logos/0162/2481/logo.gif?ixlib=rails-4.0.0&w=50&h=50&dpr=2&fit=fill&auto=compress

Title: Senior Software Engineer, Web
Company: Speechify Inc
Location: Anywhere in the World
Posted: 6d
Link: https://weworkremotely.com/company/speechify-inc
Image: No image

Title: Senior Frontend Engineer
Company: Chameleon
Location: Othe